In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/celebrity_face/train_small'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
import zipfile,os,glob,time
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import tensorflow
import tensorflow.keras as tf_keras
import shutil
tensorflow.compat.v1.disable_eager_execution()

In [ ]:
##Using pickle 
import pickle

pickle.dump( favorite_color, open( "save.p", "wb" ) )

In [60]:

Dataset = "train_small"
img_files = glob.glob("../input/celebrity-face/train_small/train_small/*")
print("Number of training image files is ",len(img_files))
width = 512
height = 512
start_time = time.perf_counter()
width_avg = 0
height_avg = 0
try:
    shutil.rmtree('/kaggle/working/celebrity-face/train_small_resized/')
os.makedirs('/kaggle/working/celebrity-face/train_small_resized/')
for img_file in img_files:
    img = Image.open(img_file).convert('RGB')
    if width_avg==0:
        width_avg = img.size[0]
    else:
        width_avg = (width_avg + img.size[0])/2
    if height_avg==0:
        height_avg = img.size[1]
    else:
        height_avg = (height_avg + img.size[1])/2
    ##Processing1:Making image size uniform
    #try:
    #print("Img size before resizing ",img.size)
    new_img=img.resize((width,height),Image.ANTIALIAS)  
    #print("Img size after resizing ",new_img.size)
    new_img.save("/kaggle/working/celebrity-face/train_small_resized/"+img_file.split("/")[-1])
    #except Exception as e:
    #    print(e)
    ##End Processing1
    
    #print(new_img.size)
print("Processing time is ",time.perf_counter() - start_time)
print("Average height is ",height_avg)
print("Average width is ",width_avg)



Number of training image files is  6955


KeyboardInterrupt: 

In [32]:
cat_data=pd.read_csv('../input/celebrity-face/category.csv')
#cat_data.head(10)
train_data = pd.read_csv('../input/celebrity-face/train_small.csv')
#train_data = train_data.iloc[0:,:]
print(train_data.head(1))
y = train_data.Category
y_id_np = np.zeros(y.size)
count = 0;
for cat in y:
    y_id_np[count] = cat_data.loc[cat_data['Category']==cat].iloc[0,0]
y_id = pd.Series(y_id_np,copy=False)
x = train_data.drop('Category',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y_id,test_size=0.2,random_state=9)
print("x is ",x.shape)
print("x_train is ",x_train.shape)
print("y_train is ",y_train.shape)
print("x_test is ",x_test.shape)
print("y_test is ",y_test.shape)
print(x_train.head(1))
#y_id = cat_data[cat_data['Category'].str.contains('Audrey Tautou')].iloc[0,0]
#print(cat_data.loc[cat_data['Category']==train_data.Category].iloc[0,0])

   Unnamed: 0 File Name       Category
0           0     0.jpg  Audrey Tautou
x is  (6955, 2)
x_train is  (5564, 2)
y_train is  (5564,)
x_test is  (1391, 2)
y_test is  (1391,)
      Unnamed: 0 File Name
5843        5843  5843.jpg


In [35]:
model = Sequential()
#model.add(tf_keras.Input(shape=))
model.add(Dense(5564, input_shape=(2,), activation='relu'))
model.add(Dense(5564/2, activation='relu'))
model.add(Dense(100, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train_bin = np_utils.to_categorical(y_train,100)
y_test_bin = np_utils.to_categorical(y_test,100)
print("Starting image reading")
model.fit(x_train, y_train_bin, validation_data=(x_test, y_test_bin), epochs=100, batch_size=2)


Train on 5564 samples, validate on 1391 samples


2022-03-21 23:57:15.574618: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100


ValueError: could not convert string to float: '5285.jpg'

In [ ]:
#Evaluation
scores = model.evaluate(X_test, Y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))